### Codigo principal

#### Instalando todas as bibliotecas necessárias

In [1]:
pip install python-dotenv langchain langchain-core langchain-ollama langchain-community langchain-chroma chromadb ollama pypdf

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
from langchain_chroma.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

embeddings = OllamaEmbeddings(
  model = "mxbai-embed-large", # Default value
)

In [3]:
CAMINHO_DB = "db"
PASTA_PDFS = "base"

prompt_template = """
Responda a pergunta do usuário:
{pergunta}
Usando o contexto a seguir:
{base_de_conhecimento}
Se você não souber a resposta, responda com "Desculpe, não sei a resposta para isso."    
"""

In [ ]:
def perguntar():
    pergunta = input("Digite sua pergunta: ")
    db = Chroma(persist_directory = CAMINHO_DB, embedding_function = embeddings)
    resultados = db.similarity_search_with_relevance_scores(pergunta, k = 4)
    if(len(resultados) == 0) or (resultados[0][1] < 0.65):
        print("Desculpe, não sei a resposta para isso.")
        return
    lista_resultados = []
    for resultado in resultados:
        texto = resultado[0].page_content
        lista_resultados.append(texto)
        
    base_de_conhecimento = "\n\n------------\n\n".join(lista_resultados)
    prompt = ChatPromptTemplate.from_template(prompt_template)
    prompt = prompt.format(pergunta=pergunta, base_de_conhecimento=base_de_conhecimento)
    modelo = ChatOllama(model="llama3.2")
    resposta = modelo.invoke(prompt).content
    print(resposta)

In [13]:
perguntar()

Pergunta: Quem são os Socializadores?

Resposta: Os Socializadores são aqueles que buscam interagir com outras pessoas e criam conexões sociais. Eles gostam de estar conectados uns com os outros e se interessam por partes do sistema que os ajudam a realizar isso.
